# Preprocessing
----------------------

This part of the kernel focuses on preprocessing the data

In [40]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import random
from datetime import datetime
from sklearn import linear_model 
from sklearn.metrics import mean_absolute_error
import plotly

# import the relevant Keras modules
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

## Cleaning and processing data

In [41]:
DATA_PATH = '/run/media/sphericalkat/archive/datasets/city_temperature/city_temperature.csv'
data = pd.read_csv(DATA_PATH, 
                   dtype={'States': 'string'}, 
                   low_memory=False,
                  )

# Drop columns we won't be using
data.drop(['Region', 'Country', 'State'], inplace=True, axis=1)

# Let's train on one city
input_data = data[data['City'] == 'Chicago']
input_data['Date'] = pd.to_datetime(input_data[['Year', 'Month', 'Day']])
input_data.drop(['Month', 'Day', 'Year'], inplace=True, axis=1)
input_data.head()

,City,AvgTemperature,Date
1831781,Chicago,23.7,1995-01-01
1831782,Chicago,13.8,1995-01-02
1831783,Chicago,14.1,1995-01-03
1831784,Chicago,3.8,1995-01-04
1831785,Chicago,5.0,1995-01-05


In [7]:
## Visualization
